In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# Case Study: Dream-Trained Navigation for Autonomous Warehouse Robots
## Implementation Notebook

This notebook implements a World Model (Ha and Schmidhuber, 2018) for the CarRacing-v2 environment, framed as an autonomous warehouse robot navigation task. You will build the complete V-M-C architecture: a Variational Autoencoder (VAE) for visual compression, a Mixture Density Network RNN (MDN-RNN) for dynamics prediction, and a simple linear Controller trained entirely inside learned "dreams."

**Business context**: FleetPath Robotics deploys autonomous mobile robots in e-commerce fulfillment centers. Their current model-free RL pipeline requires 8-12M real-world training steps and costs \$928,000 per warehouse in collision damage. Your task is to build a world model that achieves comparable navigation performance using fewer than 50,000 real environment steps -- a 200x improvement in sample efficiency.

**What you will build**:
1. A data collection pipeline that gathers rollouts from the environment
2. A VAE that compresses 64x64 images into 32-dimensional latent codes
3. An MDN-RNN that predicts multiple possible futures as mixtures of Gaussians
4. A controller with only 867 parameters, trained by dreaming
5. A complete evaluation comparing the world model agent against a rule-based baseline

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://pods.vizuara.ai/courses/world-models/practice/0/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


## 0. Setup and Dependencies

In [ ]:
# Install required packages
!pip install gymnasium[box2d] torch torchvision matplotlib numpy cma -q

import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

## 3.1 Environment Setup and Data Collection

Before building a world model, we need training data. In FleetPath's scenario, this corresponds to 2-3 days of teleoperated rollouts in a new warehouse. Here, we collect rollouts from CarRacing-v2 using a random policy.

The environment provides 96x96 RGB observations. We will resize these to 64x64 and normalize pixel values to [0, 1]. The action space is continuous with three dimensions: steering ([-1, 1]), throttle ([0, 1]), and brake ([0, 1]).

In [ ]:
# Helper: preprocess a single frame
def preprocess_frame(frame):
    """Resize to 64x64 and normalize to [0, 1]."""
    img = Image.fromarray(frame)
    img = img.resize((64, 64), Image.BILINEAR)
    return np.array(img, dtype=np.float32) / 255.0

### TODO 1: Implement the data collection pipeline

In [ ]:
def collect_rollouts(env, n_rollouts=100, max_steps=300):
    """
    Collect rollout data from the environment using a random policy.

    Args:
        env: Gymnasium environment instance
        n_rollouts: Number of episodes to collect
        max_steps: Maximum steps per episode

    Returns:
        observations: List of arrays, each of shape (T, 64, 64, 3)
        actions: List of arrays, each of shape (T, 3)

    Hints:
        1. For each rollout, reset the environment and collect frames
        2. Sample random actions using env.action_space.sample()
        3. Preprocess each frame using preprocess_frame()
        4. Store observations and actions for each episode separately
        5. Aim for ~10,000 total frames across all rollouts
    """
    # TODO: Implement this function
    pass

In [ ]:
# Collect the data
env = gym.make('CarRacing-v2', render_mode='rgb_array')
observations, actions = collect_rollouts(env, n_rollouts=100, max_steps=300)
total_frames = sum(len(obs) for obs in observations)
print(f"Collected {len(observations)} rollouts, {total_frames} total frames")

In [ ]:
# Verification
assert len(observations) > 0, "No rollouts collected"
assert observations[0].shape[1:] == (64, 64, 3), \
    f"Expected frame shape (64, 64, 3), got {observations[0].shape[1:]}"
assert 0.0 <= observations[0].min() and observations[0].max() <= 1.0, \
    "Pixel values should be in [0, 1]"
assert actions[0].shape[1] == 3, \
    f"Expected action dimension 3, got {actions[0].shape[1]}"
print("All verification checks passed.")

**Thought questions:**
- Why do we use a random policy for data collection instead of an expert policy?
- How does the quality of the collected data affect the world model's accuracy?
- In FleetPath's scenario, why is teleoperation preferred over random exploration for initial data collection?

## 3.2 Exploratory Data Analysis

Before training any model, we need to understand the structure of our data. This mirrors the data audit that FleetPath's ML team would conduct on teleoperation logs.

### TODO 2: Visualize and analyze the collected data

In [ ]:
def analyze_rollout_data(observations, actions):
    """
    Perform exploratory data analysis on the collected rollout data.

    Args:
        observations: List of observation arrays from collect_rollouts
        actions: List of action arrays from collect_rollouts

    Tasks:
        1. Plot a grid of 16 sample frames from different rollouts
           to visualize the variety of scenes
        2. Plot histograms of each action dimension (steering, throttle,
           brake) across the entire dataset
        3. Compute and plot the mean pixel difference between consecutive
           frames -- this tells us how quickly the scene changes
        4. Report summary statistics: total frames, mean episode length,
           action value ranges

    Hints:
        - Use matplotlib with a 4x4 subplot grid for the frame samples
        - For consecutive frame differences, compute
          np.mean(np.abs(obs[t+1] - obs[t])) for each t
        - The action distribution will be uniform (since we used a random
          policy) -- note this and consider how it would differ with an
          expert policy
    """
    # TODO: Implement this function
    pass

In [ ]:
# Run the analysis
analyze_rollout_data(observations, actions)

**Thought questions:**
- What does the frame difference distribution tell us about the temporal dynamics of the environment?
- If the action distribution is uniform, does that mean every region of the state space is equally well-covered? Why or why not?
- What biases might exist in FleetPath's teleoperation data that a random policy would not have?

## 3.3 Baseline: Rule-Based Proportional Controller

Before building a complex world model, we need a baseline to compare against. FleetPath's warehouse robots originally used a PID controller that steered toward detected aisle centerlines. We implement a simpler version: a rule-based controller that steers based on the position of road pixels in the image.

### TODO 3: Implement the rule-based baseline

In [ ]:
def rule_based_controller(observation):
    """
    A simple baseline controller that steers based on road position.

    Args:
        observation: A single frame of shape (64, 64, 3), values in [0, 1]

    Returns:
        action: numpy array of shape (3,) -- [steering, throttle, brake]

    Algorithm:
        1. Extract the bottom third of the image (where the road is
           closest to the car)
        2. Create a binary mask of road pixels (road is gray/dark,
           grass is green)
        3. Compute the centroid of the road pixels
        4. Set steering proportional to how far the centroid is from
           the image center
        5. Set throttle to a constant value (e.g., 0.3)
        6. Set brake to 0

    Hints:
        - Road pixels have similar R, G, B values (gray) while grass
          pixels have high G relative to R and B
        - A simple threshold: pixel is road if
          abs(R - G) < 0.1 and abs(G - B) < 0.1
        - Steering = K_p * (centroid_x - image_center_x) / image_width
          where K_p is a proportional gain (~0.5 to 1.0)
    """
    # TODO: Implement this function
    pass

In [ ]:
def evaluate_controller(env, controller_fn, n_episodes=5, max_steps=1000):
    """
    Evaluate a controller function on the environment.

    Args:
        env: Gymnasium environment instance
        controller_fn: Function that takes an observation and returns an action
        n_episodes: Number of evaluation episodes
        max_steps: Maximum steps per episode

    Returns:
        mean_reward: Average total reward across episodes
        std_reward: Standard deviation of rewards
        episode_rewards: List of per-episode rewards

    Hints:
        1. For each episode, reset the environment
        2. At each step, preprocess the observation and call controller_fn
        3. Accumulate the reward
        4. Return statistics across episodes
    """
    # TODO: Implement this function
    pass

In [ ]:
# Evaluate the baseline
baseline_mean, baseline_std, baseline_rewards = evaluate_controller(
    env, rule_based_controller, n_episodes=5
)
print(f"Baseline: {baseline_mean:.1f} +/- {baseline_std:.1f}")

In [ ]:
# Verification
assert 100 < baseline_mean < 500, \
    f"Baseline reward {baseline_mean:.1f} is outside expected range [100, 500]. " \
    "Check your road detection logic."
print(f"Baseline verification passed: reward = {baseline_mean:.1f}")

**Thought questions:**
- Why does the rule-based controller fail on sharp turns?
- What information is the rule-based controller missing that a world model agent would have?
- In what warehouse scenarios would a simple PID controller actually be sufficient?

## 3.4 Model Design: The World Model Architecture

Now we build the three-component world model: Vision (VAE), Memory (MDN-RNN), and Controller. This is the core of the case study, directly implementing the architecture from Ha and Schmidhuber (2018).

The key insight: **the controller never sees raw pixels.** It lives entirely in a compressed "dream space" created by V and M. All the complexity of the visual world is handled by the vision and memory components, so the controller can be incredibly simple (867 parameters).

### TODO 4a: Implement the VAE

In [ ]:
class VAE(nn.Module):
    """
    Variational Autoencoder for visual compression.

    Compresses 64x64x3 images into 32-dimensional latent codes.

    Architecture:
        Encoder: 4 convolutional layers (stride 2) reducing spatial
                 dimensions 64 -> 30 -> 14 -> 6 -> 2, followed by
                 two linear heads for mu and logvar
        Decoder: Linear layer expanding to 2x2x256, followed by 4
                 transposed convolutional layers restoring to 64x64x3

    Key concepts to implement:
        - The encoder outputs parameters of a distribution (mu, logvar),
          NOT a deterministic code
        - The reparameterization trick: z = mu + std * epsilon
        - The decoder uses transposed convolutions (the "reverse" of
          convolutions)
    """

    def __init__(self, latent_dim=32):
        super().__init__()
        # TODO: Define encoder layers
        # Hint: Use nn.Conv2d with kernel_size=4, stride=2
        # Layer sizes: 3->32->64->128->256 channels
        # After convolutions, flatten and project to mu and logvar

        # TODO: Define decoder layers
        # Hint: Use nn.ConvTranspose2d to reverse the encoder
        # First project from latent_dim to 256*2*2 with a linear layer
        # Then use transposed convolutions: 256->128->64->32->3 channels
        # Use kernel sizes [5, 5, 6, 6] and stride 2 for the decoder
        # Final activation should be Sigmoid (pixel values in [0,1])
        pass

    def encode(self, x):
        """
        Encode an image to latent distribution parameters.

        Args:
            x: Image tensor of shape (batch, 3, 64, 64)

        Returns:
            mu: Mean of shape (batch, latent_dim)
            logvar: Log-variance of shape (batch, latent_dim)
        """
        # TODO: Pass x through encoder convolutions, flatten,
        # project to mu and logvar
        pass

    def reparameterize(self, mu, logvar):
        """
        Sample from the latent distribution using the reparameterization trick.

        Args:
            mu: Mean of shape (batch, latent_dim)
            logvar: Log-variance of shape (batch, latent_dim)

        Returns:
            z: Sampled latent code of shape (batch, latent_dim)

        Key insight: z = mu + std * epsilon where epsilon ~ N(0, I)
        std = exp(0.5 * logvar) because logvar = log(sigma^2),
        so exp(0.5 * logvar) = sigma.
        """
        # TODO: Implement the reparameterization trick
        pass

    def decode(self, z):
        """
        Decode a latent code back to an image.

        Args:
            z: Latent code of shape (batch, latent_dim)

        Returns:
            x_recon: Reconstructed image of shape (batch, 3, 64, 64)
        """
        # TODO: Project z to spatial dimensions, apply transposed convolutions
        pass

    def forward(self, x):
        """Full forward pass: encode, sample, decode."""
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

### TODO 4b: Implement the MDN-RNN

In [ ]:
class MDNRNN(nn.Module):
    """
    Mixture Density Network RNN for dynamics prediction.

    Takes a sequence of (latent_code, action) pairs and predicts
    the distribution of the next latent code as a mixture of Gaussians.

    Architecture:
        - LSTM with hidden_dim=256
        - Three linear heads projecting hidden state to:
          pi (mixing coefficients), mu (means), sigma (std devs)
          for K=5 Gaussian components

    Key concepts:
        - The LSTM hidden state h_t serves as the agent's memory
        - The MDN head outputs parameters for K Gaussians, allowing
          multimodal predictions (multiple possible futures)
        - pi values are passed through softmax to form a valid
          probability distribution
        - sigma values are exponentiated to ensure positivity
    """

    def __init__(self, latent_dim=32, action_dim=3,
                 hidden_dim=256, n_gaussians=5):
        super().__init__()
        self.latent_dim = latent_dim
        self.n_gaussians = n_gaussians
        self.hidden_dim = hidden_dim

        # TODO: Define the LSTM layer
        # Input size: latent_dim + action_dim (concatenated)
        # Hidden size: hidden_dim
        # Use batch_first=True

        # TODO: Define three linear heads for pi, mu, sigma
        # Each outputs: latent_dim * n_gaussians values
        pass

    def forward(self, z, a, hidden=None):
        """
        Predict the distribution of the next latent state.

        Args:
            z: Latent codes, shape (batch, seq_len, latent_dim)
            a: Actions, shape (batch, seq_len, action_dim)
            hidden: Optional LSTM hidden state tuple

        Returns:
            pi: Mixing coefficients (batch*seq_len, latent_dim, K)
            mu: Means (batch*seq_len, latent_dim, K)
            sigma: Std devs (batch*seq_len, latent_dim, K)
            hidden: Updated LSTM hidden state

        Steps:
            1. Concatenate z and a along the last dimension
            2. Pass through LSTM to get hidden states
            3. Reshape LSTM output for the MDN heads
            4. Apply softmax to pi (mixing coefficients must sum to 1)
            5. Apply exp to sigma (std devs must be positive)
        """
        # TODO: Implement the forward pass
        pass

### TODO 4c: Implement the Controller

In [ ]:
class Controller(nn.Module):
    """
    Simple linear controller.

    Takes the concatenation of the current latent code z_t and the
    LSTM hidden state h_t, and outputs an action.

    Total parameters: (latent_dim + hidden_dim) * action_dim + action_dim
                    = (32 + 256) * 3 + 3 = 867
    """

    def __init__(self, latent_dim=32, hidden_dim=256, action_dim=3):
        super().__init__()
        # TODO: Define a single linear layer
        # Input: latent_dim + hidden_dim
        # Output: action_dim
        pass

    def forward(self, z, h):
        """
        Choose an action given the current state representation.

        Args:
            z: Current latent code, shape (batch, latent_dim)
            h: Current LSTM hidden state, shape (batch, hidden_dim)

        Returns:
            action: Action vector, shape (batch, action_dim)
                    Values in [-1, 1] (use tanh activation)
        """
        # TODO: Concatenate z and h, pass through linear layer, apply tanh
        pass

In [ ]:
# Verification: check parameter counts
vae = VAE().to(device)
mdnrnn = MDNRNN().to(device)
controller = Controller().to(device)

for name, model in [("VAE", vae), ("MDNRNN", mdnrnn),
                     ("Controller", controller)]:
    n_params = sum(p.numel() for p in model.parameters())
    print(f"{name}: {n_params:,} parameters")

assert sum(p.numel() for p in controller.parameters()) == 867, \
    "Controller should have exactly 867 parameters"
print("\nController parameter count verified: 867")

**Thought questions:**
- Why does the controller use tanh activation but the VAE decoder uses sigmoid?
- What would happen if we made the controller a 3-layer MLP instead of a single linear layer?
- Why is the MDN-RNN's hidden dimension (256) much larger than the latent dimension (32)?

## 3.5 Training Strategy

Training happens in three sequential stages, mirroring FleetPath's deployment pipeline: first learn to see (V), then learn to predict (M), then learn to act (C).

### Stage 1: Train the VAE

The VAE loss has two terms:
- **Reconstruction loss**: MSE between input and reconstructed image
- **KL divergence**: Regularizes the latent space toward a standard Gaussian

### TODO 5a: Implement the VAE training loop

In [ ]:
def vae_loss(x_recon, x, mu, logvar, beta=1.0):
    """
    Compute the VAE loss: reconstruction + KL divergence.

    Args:
        x_recon: Reconstructed image, shape (batch, 3, 64, 64)
        x: Original image, shape (batch, 3, 64, 64)
        mu: Encoder mean, shape (batch, latent_dim)
        logvar: Encoder log-variance, shape (batch, latent_dim)
        beta: KL weight (beta=1 is standard VAE, beta<1 sharper reconstructions)

    Returns:
        loss: Scalar loss value
        recon_loss: Reconstruction component (for logging)
        kl_loss: KL divergence component (for logging)

    Formulas:
        Reconstruction loss = MSE(x_recon, x) summed over pixels, averaged over batch
        KL loss = -0.5 * sum(1 + logvar - mu^2 - exp(logvar)), averaged over batch

    Hints:
        - Use F.mse_loss with reduction='sum' and divide by batch size
        - The KL formula is the closed-form KL divergence between N(mu, sigma^2) and N(0, 1)
        - Sum the KL over latent dimensions, then average over the batch
    """
    # TODO: Implement the VAE loss function
    pass

In [ ]:
def train_vae(vae, train_data, n_epochs=10, batch_size=64, lr=1e-3):
    """
    Train the VAE on collected observation frames.

    Args:
        vae: VAE model instance
        train_data: Tensor of frames, shape (N, 3, 64, 64)
        n_epochs: Number of training epochs
        batch_size: Batch size
        lr: Learning rate

    Returns:
        vae: Trained VAE model
        losses: List of per-epoch average losses

    Steps:
        1. Create a DataLoader from train_data
        2. Use Adam optimizer with the specified learning rate
        3. For each epoch, iterate over batches:
           a. Forward pass through the VAE
           b. Compute the VAE loss
           c. Backpropagate and update weights
        4. Log reconstruction loss and KL loss separately
        5. Every 2 epochs, visualize 8 original vs. reconstructed
           images side by side

    Hints:
        - Use beta=1.0 initially; if reconstructions are too blurry, try beta=0.5
        - Learning rate 1e-3 with Adam works well for this scale
    """
    # TODO: Implement the training loop
    pass

In [ ]:
# Prepare training data: stack all frames into a single tensor
all_frames = np.concatenate(observations, axis=0)  # (N, 64, 64, 3)
all_frames_tensor = torch.FloatTensor(all_frames).permute(0, 3, 1, 2).to(device)
print(f"Training data shape: {all_frames_tensor.shape}")

# Train the VAE
vae, vae_losses = train_vae(vae, all_frames_tensor, n_epochs=10)

In [ ]:
# Visualize training progress
plt.figure(figsize=(8, 4))
plt.plot(vae_losses, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('VAE Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

### Stage 2: Train the MDN-RNN

With the VAE trained, we encode all observations into latent codes and train the MDN-RNN to predict the next latent state given the current state and action.

### TODO 5b: Implement the MDN-RNN training loop

In [ ]:
def mdn_loss(pi, mu, sigma, z_next):
    """
    Compute the MDN negative log-likelihood loss.

    Args:
        pi: Mixing coefficients, shape (batch, latent_dim, K)
        mu: Means, shape (batch, latent_dim, K)
        sigma: Std devs, shape (batch, latent_dim, K)
        z_next: Actual next latent state, shape (batch, latent_dim)

    Returns:
        loss: Scalar negative log-likelihood

    Steps:
        1. Expand z_next to match the K dimension
        2. Compute Gaussian log-probability for each component
        3. Weight by mixing coefficients (in log space)
        4. Use logsumexp across K for numerical stability
        5. Sum over latent dimensions, average over batch

    Hints:
        - Use torch.logsumexp for numerical stability
        - log N(x|mu,sigma) = -0.5*((x-mu)/sigma)^2 - log(sigma) - 0.5*log(2*pi)
        - Clamp sigma to min 1e-6 to prevent division by zero
    """
    # TODO: Implement the MDN loss function
    pass

In [ ]:
def train_mdnrnn(mdnrnn, vae, rollout_obs, rollout_actions,
                  n_epochs=20, seq_len=32, batch_size=16, lr=1e-3):
    """
    Train the MDN-RNN on encoded rollout sequences.

    Args:
        mdnrnn: MDNRNN model instance
        vae: Trained VAE model (frozen, used only for encoding)
        rollout_obs: List of observation arrays from rollouts
        rollout_actions: List of action arrays from rollouts
        n_epochs: Number of training epochs
        seq_len: Sequence length for LSTM training
        batch_size: Batch size
        lr: Learning rate

    Returns:
        mdnrnn: Trained MDNRNN model
        losses: List of per-epoch average losses

    Steps:
        1. Encode all observations into latent codes using the trained VAE
           (with torch.no_grad)
        2. Create training sequences of length seq_len from the encoded rollouts
        3. For each epoch, iterate over batches of sequences:
           a. Feed (z_t, a_t) for t=0..T-1 into the MDN-RNN
           b. Compute MDN loss against z_{t+1} for t=0..T-1
           c. Backpropagate and update weights

    Hints:
        - Freeze the VAE during this stage (vae.eval(), no_grad)
        - Use truncated backpropagation: detach hidden states between sequences
        - Learning rate 1e-3 with Adam, gradient clipping at 1.0
    """
    # TODO: Implement the training loop
    pass

In [ ]:
# Train the MDN-RNN
mdnrnn, mdnrnn_losses = train_mdnrnn(
    mdnrnn, vae, observations, actions, n_epochs=20
)

In [ ]:
# Visualize training progress
plt.figure(figsize=(8, 4))
plt.plot(mdnrnn_losses, marker='o')
plt.xlabel('Epoch')
plt.ylabel('MDN-RNN Loss (NLL)')
plt.title('MDN-RNN Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

### Stage 3: Train the Controller in Dreams

This is the key insight of the World Model architecture. The controller is trained entirely inside the learned world model -- it never touches the real environment during training. We use CMA-ES (Covariance Matrix Adaptation Evolution Strategy) to evolve the controller's 867 parameters.

### TODO 5c: Implement dream-based controller training

In [ ]:
def dream_rollout(vae, mdnrnn, controller, initial_z, max_steps=200):
    """
    Run a rollout entirely inside the learned world model (the dream).

    Args:
        vae: Trained VAE (used only for initial encoding)
        mdnrnn: Trained MDN-RNN
        controller: Controller to evaluate
        initial_z: Starting latent code, shape (latent_dim,)
        max_steps: Maximum dream steps

    Returns:
        total_reward: Estimated cumulative reward from the dream
        dream_states: List of dreamed latent states

    Steps:
        1. Initialize h_0 (LSTM hidden state) as zeros
        2. Set z_0 = initial_z
        3. For each step t:
           a. Get action from controller: a_t = C(z_t, h_t)
           b. Predict next state distribution: pi, mu, sigma, h_{t+1} = M(z_t, a_t, h_t)
           c. Sample z_{t+1} from the predicted mixture
           d. Estimate reward (heuristic: reward = -|steering| + throttle)
        4. Return total reward

    Hints:
        - For sampling from the mixture: first choose component k with
          probability pi_k, then sample from N(mu_k, sigma_k^2)
        - For reward estimation, use: reward = throttle - 0.1 * |steering|
        - Keep all tensors on the same device
    """
    # TODO: Implement dream rollouts
    pass

In [ ]:
def train_controller_cmaes(vae, mdnrnn, controller,
                            initial_observations, n_generations=50,
                            population_size=32, max_dream_steps=200):
    """
    Train the controller using CMA-ES inside the learned dream.

    Args:
        vae: Trained VAE
        mdnrnn: Trained MDN-RNN
        controller: Controller to train
        initial_observations: Real observations to seed dreams
        n_generations: Number of CMA-ES generations
        population_size: Number of controllers per generation
        max_dream_steps: Steps per dream rollout

    Returns:
        controller: Trained controller
        fitness_history: List of best fitness per generation

    Steps:
        1. Extract initial controller parameters as a flat vector
        2. Initialize CMA-ES with these parameters
        3. For each generation:
           a. Sample population_size parameter vectors
           b. For each vector, load into controller, run dream_rollout
           c. Record fitness (total reward)
           d. Update CMA-ES distribution
           e. Log best fitness
        4. Load best parameters into controller

    Hints:
        - Install cma: already done in setup
        - Use cma.CMAEvolutionStrategy(initial_params, sigma0)
          with sigma0 = 0.1
        - Use multiple dream rollouts per controller (3-5) and
          average for stable evaluation
    """
    # TODO: Implement CMA-ES training
    pass

In [ ]:
# Encode initial observations for seeding dreams
with torch.no_grad():
    sample_frames = all_frames_tensor[:100]
    initial_mu, _ = vae.encode(sample_frames)

# Train the controller in dreams
controller, fitness_history = train_controller_cmaes(
    vae, mdnrnn, controller, initial_mu, n_generations=50
)

In [ ]:
# Plot CMA-ES training progress
plt.figure(figsize=(8, 4))
plt.plot(fitness_history, marker='o', markersize=3)
plt.xlabel('Generation')
plt.ylabel('Best Fitness (Dream Reward)')
plt.title('Controller Training via CMA-ES (Inside the Dream)')
plt.grid(True, alpha=0.3)
plt.show()

**Thought questions:**
- Why do we train V, M, and C sequentially rather than end-to-end?
- What happens if the world model (M) is inaccurate? How would this affect the controller?
- The Dreamer algorithm (2020) replaces CMA-ES with backpropagation through the dream. What are the advantages and disadvantages of each approach?

## 3.6 Evaluation

After training, we evaluate the world model agent in the *real* environment (not the dream) and compare against the rule-based baseline. This is the moment of truth -- does a controller trained entirely in dreams actually work in reality?

### TODO 6: Evaluate the trained world model agent

In [ ]:
def evaluate_world_model_agent(env, vae, mdnrnn, controller,
                                n_episodes=10, max_steps=1000,
                                render=False):
    """
    Evaluate the trained world model agent in the real environment.

    Args:
        env: Gymnasium environment instance
        vae: Trained VAE
        mdnrnn: Trained MDN-RNN
        controller: Trained Controller
        n_episodes: Number of evaluation episodes
        max_steps: Maximum steps per episode
        render: Whether to save rendered frames

    Returns:
        results: Dictionary containing:
            - mean_reward: Average reward across episodes
            - std_reward: Standard deviation
            - episode_rewards: List of per-episode rewards
            - episode_lengths: List of episode lengths
            - frames: List of rendered frames (if render=True)

    Steps:
        1. For each episode:
           a. Reset environment, get initial observation
           b. Encode initial observation with VAE: z_0 = V(x_0)
           c. Initialize LSTM hidden state: h_0 = zeros
           d. At each step:
              - Get action: a_t = C(z_t, h_t)
              - Step the REAL environment: x_{t+1}, r_t = env.step(a_t)
              - Encode new observation: z_{t+1} = V(x_{t+1})
              - Update LSTM: _, _, _, h_{t+1} = M(z_t, a_t, h_t)
              - Accumulate reward
           e. Record episode reward and length
        2. Compute and return statistics

    Hints:
        - Set all models to eval mode and use torch.no_grad()
        - The controller outputs values in [-1, 1] via tanh;
          map throttle and brake to [0, 1] using (action + 1) / 2
        - Remember to preprocess observations (resize, normalize)
    """
    # TODO: Implement evaluation
    pass

In [ ]:
# Evaluate the world model agent
wm_results = evaluate_world_model_agent(env, vae, mdnrnn, controller, n_episodes=10)
print(f"World Model Agent: {wm_results['mean_reward']:.1f} +/- {wm_results['std_reward']:.1f}")
print(f"Baseline:          {baseline_mean:.1f} +/- {baseline_std:.1f}")

In [ ]:
def plot_comparison(baseline_rewards, wm_rewards):
    """
    Create a comparison plot of baseline vs. world model performance.

    Args:
        baseline_rewards: List of episode rewards for the baseline
        wm_rewards: List of episode rewards for the world model agent

    Tasks:
        1. Create a bar chart comparing mean rewards with error bars
        2. Create a box plot showing the distribution of rewards
        3. Print a summary table with mean, std, min, max for each

    Hints:
        - Use matplotlib subplots with 1 row, 2 columns
        - Error bars should show standard deviation
        - Include a horizontal line at reward=900 labeled "Human-level reference"
    """
    # TODO: Implement comparison visualization
    pass

plot_comparison(baseline_rewards, wm_results['episode_rewards'])

**Thought questions:**
- How does the agent's performance in the real environment compare to its performance in dreams? What explains the gap?
- If the world model had perfect fidelity, would the dream performance and real performance be identical? Why or why not?
- What is the sample efficiency ratio? How many real environment steps did the world model agent use compared to the model-free baseline's 10M steps?

## 3.7 Error Analysis

Understanding failure modes is critical for FleetPath's safety case. A robot that performs well on average but catastrophically fails in specific scenarios is not deployable.

### TODO 7: Analyze failure modes

In [ ]:
def failure_analysis(env, vae, mdnrnn, controller,
                      n_episodes=20, max_steps=1000):
    """
    Identify and categorize failure modes of the trained agent.

    Args:
        env: Gymnasium environment
        vae, mdnrnn, controller: Trained model components
        n_episodes: Number of episodes to analyze
        max_steps: Maximum steps per episode

    Returns:
        failures: List of dicts with failure details
        summary: Dict with counts per failure category

    Failure categories:
        1. "off_track" -- car leaves the road (reward < -1 for 3+ consecutive steps)
        2. "stall" -- car speed drops to near zero (< 10 steps of low reward)
        3. "oscillation" -- rapid left-right steering (sign changes > 5 in 10 steps)
        4. "model_exploitation" -- dream reward >> real reward for same trajectory

    Steps:
        1. Run episodes recording frame-by-frame data (actions, rewards, latents)
        2. Detect failures using the heuristics above
        3. For each failure, save the context (step number, type, observation)
        4. Summarize: count failures by category, compute overall failure rate

    Hints:
        - Track the running reward to detect off_track events
        - Track action history to detect oscillation (np.diff on steering sign)
        - Compare dream-predicted z with actual encoded z for model exploitation
    """
    # TODO: Implement failure analysis
    pass

In [ ]:
# Run failure analysis
failures, failure_summary = failure_analysis(env, vae, mdnrnn, controller)
print("Failure Summary:")
for category, count in failure_summary.items():
    print(f"  {category}: {count}")

**Thought questions:**
- Which failure modes are most concerning for warehouse deployment? Why?
- How could the world model be improved to reduce model exploitation failures?
- If you were FleetPath's safety engineer, what minimum failure rate would you require before approving deployment?

## 3.8 Scalability and Deployment Considerations

FleetPath needs to deploy this model on NVIDIA Jetson AGX Orin with under 100ms inference latency. Let us profile the inference pipeline to verify feasibility.

### TODO 8: Profile inference performance

In [ ]:
def profile_inference(vae, mdnrnn, controller, device='cpu'):
    """
    Profile the inference latency of each component.

    Args:
        vae: Trained VAE
        mdnrnn: Trained MDN-RNN
        controller: Trained Controller
        device: Device to profile on ('cpu' or 'cuda')

    Returns:
        timing: Dictionary with per-component latency in ms

    Steps:
        1. Create a dummy input (random 64x64x3 image)
        2. Time 100 forward passes through each component:
           a. VAE encoder: x -> z
           b. MDN-RNN: (z, a) -> (pi, mu, sigma, h)
           c. Controller: (z, h) -> a
           d. Full pipeline: x -> z -> (z, h) -> a
        3. Report mean and std of latency for each component
        4. Determine if the 100ms (10 Hz) budget is met

    Hints:
        - Use torch.cuda.synchronize() before timing on GPU
        - Warm up with 10 untimed passes before measuring
        - Use time.perf_counter() for precise timing
    """
    # TODO: Implement inference profiling
    pass

In [ ]:
# Profile on available device
timing = profile_inference(vae, mdnrnn, controller, device=str(device))
for component, ms in timing.items():
    print(f"  {component}: {ms:.2f} ms")

total = timing.get('full_pipeline', sum(timing.values()))
print(f"\n  Total: {total:.2f} ms")
print(f"  Budget: 100 ms")
print(f"  Status: {'PASS' if total < 100 else 'FAIL'}")

**Thought questions:**
- Which component is the bottleneck? Why?
- How would you reduce latency if the 100ms budget is not met?
- What is the trade-off between model size and dream fidelity?

## 3.9 Ethical and Regulatory Analysis

FleetPath's robots operate alongside human workers. Safety, fairness, and regulatory compliance are not optional -- they are prerequisites for deployment.

### TODO 9: Ethical impact assessment

In [ ]:
def ethical_assessment():
    """
    Write a brief ethical impact assessment for deploying a
    world-model-based navigation system in a fulfillment center.

    Address these five areas (write your answers as a formatted
    string and print it):

    1. SAFETY: The world model may have inaccurate dynamics in some
       regions. How should the system detect and respond when predictions
       diverge from reality? What fallback mechanisms should exist?

    2. WORKER IMPACT: Autonomous robots change warehouse work. What
       safeguards ensure worker safety? How should the system handle
       edge cases where a human is in an unexpected position?

    3. BIAS: Training data collected during day shifts may not represent
       night shift conditions (different lighting, fewer workers). How
       should this be detected and mitigated?

    4. ACCOUNTABILITY: If a dream-trained robot causes injury, who is
       responsible? How should accountability be structured?

    5. REGULATORY: What standards or certifications would FleetPath need?
       Consider ISO 3691-4, OSHA, and emerging AI regulation.

    Return your answers as a formatted string.
    """
    # TODO: Write your ethical assessment
    assessment = """
    [Write your assessment here -- address all 5 areas above]
    """
    print(assessment)
    return assessment

ethical_assessment()

**Thought questions:**
- Should dream-trained robots be held to a higher safety standard than conventionally trained robots? Why or why not?
- How does the "model exploitation" problem interact with safety requirements?
- If you discovered that the world model consistently underestimates the speed of human workers, what would you do?

## Summary

In this notebook, you built a complete World Model system for autonomous navigation:

1. **Data Collection** -- Gathered rollout data from the environment (50,000 frames vs. 10M for model-free RL)
2. **Vision (V)** -- Trained a VAE to compress 64x64 images into 32-dimensional latent codes (384:1 compression)
3. **Memory (M)** -- Trained an MDN-RNN to predict multiple possible futures as mixtures of Gaussians
4. **Controller (C)** -- Trained a 867-parameter linear controller entirely inside learned "dreams" using CMA-ES
5. **Evaluation** -- Compared the dream-trained agent against a rule-based baseline in the real environment
6. **Error Analysis** -- Identified and categorized failure modes for deployment safety
7. **Deployment** -- Profiled inference latency against the 100ms production budget
8. **Ethics** -- Assessed the societal implications of deploying dream-trained autonomous robots

**Key takeaway**: The world model achieves comparable performance to model-free methods while using 200x fewer real-world interactions. The agent literally learns to drive by dreaming -- a paradigm that slashes FleetPath's warehouse onboarding cost from \$928,000 to under \$3,000.

For further reading on production system design (API design, A/B testing, drift detection, cost analysis), see **Section 4** of the full case study document.

### References

- Ha and Schmidhuber, "World Models" (2018)
- Hafner et al., "Dream to Control: Learning Behaviors by Latent Imagination" (Dreamer, 2020)
- Hafner et al., "Mastering Diverse Domains through World Models" (DreamerV3, 2023)
- Kingma and Welling, "Auto-Encoding Variational Bayes" (2014)